Розробка моделі, здатної ідентифікувати та класифікувати різні рівні токсичності в коментарях, використовуючи можливості BERT (Bidirectional Encoder Representations from Transformers) для аналізу тексту.

Загальний опис проблеми та підходу.

Для навчання наявна дуже незбалансована вибірка і задача з багатоміткової класифікації досить складна. Тому планується застосувати багатозадачний підхід і в моделі виконувати класифікацію у два етапи:

* бінарна класифікація: токсичні - нетоксичні коментарі
* серед токсичних коментарів: багатоміткова класифікація типу токсичності

В рамках цього модулю буде виконано:

* Підготовка датасету до передачі у модель: перетворення форматів для прийняття даних моделлю, виділення міні-вибірки для навчання тестових версій моделі і вибору оптимальної архітектури.
* Розробка і тестування архітектури моделі: на цьому етапі планується виконати декілька варіантів моделі із різною логікою, протестувати чи працює код, виконати навчання підготовлених моделей на міні-вибірці з метою вибору кращої архітектури.
* Розробка та навчання фінальної моделі із найкращими параметрами на повній тренувальній вибірці.
* Оцінка якості моделі та адекватності передбачень.



In [ ]:
# Перевірка підключення GPU
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [8]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Layer, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Precision, Recall


from transformers import TFBertForSequenceClassification, TFBertModel

import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from sklearn.model_selection import train_test_split


**Підготовка тренувальних даних**

In [2]:
data_path = '/kaggle/input/dataset-new/train_data.csv'
df = pd.read_csv(data_path)

# список категорій:
LABEL_COLUMNS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Конвертація токенізованих даних з рядків у масиви
for column in ['input_ids', 'attention_masks']:
    df[column] = df[column].apply(eval).apply(np.array)

# Виділяємо токенізовані вектори та мітки
input_ids = np.stack(df['input_ids'].values)
attention_mask = np.stack(df['attention_masks'].values)
labels = np.array(df[LABEL_COLUMNS].values)
labels = labels.astype('float32')

In [3]:
# Розділення на тренувальну та тестову вибірки (повний набір даних)

train_input_ids, val_input_ids, train_attention_mask, val_attention_mask, train_labels, val_labels = train_test_split(
    input_ids, attention_mask, labels, test_size=0.2, random_state=42
)

In [4]:
# Виділення міні-вибірки 5%

_, mini_input_ids, _, mini_attention_mask, _, mini_labels = train_test_split(
    input_ids, attention_mask, labels, test_size=0.05, random_state=42
)

In [5]:
# розділення міні вибірки на тренувальну та валідаційну
t_input_ids, v_input_ids, t_attention_mask, v_attention_mask, t_labels, v_labels = train_test_split(
    mini_input_ids, mini_attention_mask, mini_labels, test_size=0.2, random_state=42
)


In [6]:
# "Розподіл класів у повній вибірці
print("Розподіл класів у повній вибірці:\n", labels.sum(axis=0))
print("Кількість прикладів нетоксичних коментарів:", (labels.sum(axis=1) == 0).sum())

# Розподіл класів у міні вибірці
print("Розподіл класів у міні вибірці:\n", mini_labels.sum(axis=0))
print("Кількість прикладів нетоксичних коментарів:", (mini_labels.sum(axis=1) == 0).sum())

Розподіл класів у повній вибірці:
 [15294.  1595.  8449.   478.  7877.  1405.]
Кількість прикладів нетоксичних коментарів: 143346
Розподіл класів у міні вибірці:
 [748.  80. 421.  13. 410.  71.]
Кількість прикладів нетоксичних коментарів: 7181


**Підбір архітектури моделі на міні-вибірці**

Для роботи із незбалансованими даними у якості метрики доцільно використовувати F1 метрику.

Проте її використання на пряму (врахування у функції втрат) має певні проблеми і планується використовувати вбудовані функції втрат.

* метрика Ф-1 є нелінійною та залежить від precision та recall. Її оптимізація на пряму через функцію втрат вимагає складних розрахунків які можуть бути не стабільними
* спроби оптимізувати Ф1 на пряму можуть призвести до того, що градієнти стануть надто малими і навчання буде стагнувати
* тому планується додати кастомну метрику Ф-1 для відслідковування, а у якості функції втрат використовувати вбудовані функції (тобто модель при навчанні не буде її оптимізувати на пряму).
* на етапі вибору моделі пріоритет матимуть моделі які показали найкращу динаміку по метриці Ф-1
* у функції втрат планується додати ваги класів для того, щоб врахувати дисбаланс даних



В рамках цього етапу планується реалізувати 3 стратегії побудови моделі:

* Єдина модель із двома незалежними "головами" для бінарної класифікації токсичних-нетоксичних коментарів та для мультиміткової класифікації токсичних коментарів. Обидві голови будуть навчатись на повних вибірках даних. Модель буде мати занижену точність у порівнянні із фактичною за рахунок помилок другої голови на нетоксичних прикладах.
* Єдина модель аналогічна першій, проте друга модель буде навчатись лише на токсичних прикладах для більш сфокусованого навчання.
* Дві послідовні моделі, перша з яких виконає бінарну класифікацію, а друга повністю незалежно навчається лише на токсичних коментарях і виконує їх класифікацію.

Через обмеження обчислювальних ресурсів ми не будемо підбирати параметри моделей та викорастаємо "best prаctices". Також на этапі експериментів ми будемо виконувати лише feature extraction без fine-tuning для економії ресурсів.

За результатами оцінки буде вибрано кращу стратегію та на основі її побудовано фінальну модель та навчено на повній вибірці даних.


In [7]:
# Кастомний шар для інтеграції з BERT
class BertLayer(Layer):
    def __init__(self, pretrained_model_name="bert-base-uncased", trainable=False, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        # Завантажуємо попередньо навчений BERT
        self.bert = TFBertModel.from_pretrained(pretrained_model_name)
        self.bert.trainable = trainable  # Заморожуємо або розморожуємо шари залежно від параметра trainable

    def call(self, inputs):
        # Вхідні дані: input_ids та attention_mask
        input_ids, attention_mask = inputs
        # Передаємо дані через BERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state  # Повертаємо тільки last_hidden_state

In [10]:
'''
у якості метрики обрано Ф-1 у зв'язку із незбалансованістю класів. 
Підготуємо функцію для неї
'''

import tensorflow as tf
from tensorflow.keras import backend as K

def f1_metric(y_true, y_pred):
    # Преобразуем в бинарный формат для каждого класса
    y_true = K.cast(y_true, 'int32')
    y_pred = K.cast(K.greater_equal(y_pred, 0.5), 'int32')

    # Вычисляем точность (precision) и полноту (recall)
    true_positive = K.sum(K.cast(y_true * y_pred, 'float32'))
    false_positive = K.sum(K.cast((1 - y_true) * y_pred, 'float32'))
    false_negative = K.sum(K.cast(y_true * (1 - y_pred), 'float32'))

    precision = true_positive / (true_positive + false_positive + K.epsilon())
    recall = true_positive / (true_positive + false_negative + K.epsilon())

    # F1-score = 2 * (precision * recall) / (precision + recall)
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    
    return f1

In [18]:
# ваги класів 

# Розподіл класів у повній вибірці
class_distribution = np.array([15294., 1595., 8449., 478., 7877., 1405.])

# Розрахунок ваг класів
total_samples = np.sum(class_distribution)
class_weights = total_samples / (len(class_distribution) * class_distribution)

# Нормалізація ваг
class_weights = class_weights / np.min(class_weights)

# Вивід розрахованих ваг
print(f"Class weights for multilabel_output: {class_weights}")

Class weights for multilabel_output: [ 1.          9.58871473  1.81015505 31.9958159   1.94160213 10.88540925]


In [20]:
# Кастомна функція втрат для врахування ваг класів

def weighted_binary_crossentropy(class_weights):
    def loss(y_true, y_pred):
        # Применяем веса для каждого класса
        weights = tf.reduce_sum(class_weights * y_true, axis=-1)
        # Бинарная кроссэнтропия
        bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        # Взвешиваем loss
        return bce * weights
    return loss

**Перша версія моделі**

In [21]:
# Вхідні дані
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# BERT шар
bert_outputs = BertLayer(trainable=False)([input_ids, attention_mask])

# Пулінг
pooled_output = GlobalAveragePooling1D()(bert_outputs)

# Перша голова - бінарна класифікація
binary_dense = Dense(128, activation="swish")(pooled_output)
binary_dropout = Dropout(0.3)(binary_dense)
binary_output = Dense(1, activation="sigmoid", name="binary_output")(binary_dropout)

# Друга голова - багатоміткова класифікація токсичних коментарів
multilabel_dense = Dense(128, activation="swish")(pooled_output)
multilabel_dropout = Dropout(0.3)(multilabel_dense)
multilabel_output = Dense(6, activation="sigmoid", name="multilabel_output")(multilabel_dropout)

# Модель
model_1 = Model(
    inputs=[input_ids, attention_mask],
    outputs=[binary_output, multilabel_output]
)

class_weights_tensor = tf.constant(class_weights, dtype=tf.float32)

# Компіляція моделі
model_1.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss={
        "binary_output": "binary_crossentropy",
        "multilabel_output": weighted_binary_crossentropy(class_weights_tensor)
    },
    loss_weights={
        "binary_output": 0.5,
        "multilabel_output": 1.0
    },
    metrics={
        "binary_output": ["accuracy"],
        "multilabel_output": ["accuracy", Precision(name="precision"), Recall(name="recall"), f1_metric]
    }
)

model_1.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_ids (InputLayer)    │ (None, 128)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_mask            │ (None, 128)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_layer_2 (BertLayer)  │ (None, 128, 768)       │              0 │ input_ids[0][0],       │
│                           │                        │                │ attention_mask[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 768)            │              0 │ bert_layer_2[0][0]     │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 128)            │         98,432 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 128)            │         98,432 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 128)            │              0 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)       │ (None, 128)            │              0 │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ binary_output (Dense)     │ (None, 1)              │            129 │ dropout_4[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multilabel_output (Dense) │ (None, 6)              │            774 │ dropout_5[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 197,767 (772.53 KB)

 Trainable params: 197,767 (772.53 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Генерація міток для першої голови (binary_output)
t_binary_labels = np.where(np.all(t_labels == 0, axis=1), 1, 0).astype('float32')
v_binary_labels = np.where(np.all(v_labels == 0, axis=1), 1, 0).astype('float32')

print(f"Shape of t_binary_labels: {t_binary_labels.shape}")
print(f"Example t_binary_labels: {t_binary_labels[:10]}")
print(f"Shape of v_binary_labels: {v_binary_labels.shape}")
print(f"Example v_binary_labels: {v_binary_labels[:10]}")

Shape of t_binary_labels: (6383,)
Example t_binary_labels: [1. 1. 1. 0. 1. 1. 1. 1. 1. 1.]
Shape of v_binary_labels: (1596,)
Example v_binary_labels: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [23]:
# Підготовка даних для двух голов
t_data = {
    "input_ids": t_input_ids,
    "attention_mask": t_attention_mask,
}

v_data = {
    "input_ids": v_input_ids,
    "attention_mask": v_attention_mask,
}

t_labels_combined = {
    "binary_output": t_binary_labels,
    "multilabel_output": t_labels,
}

v_labels_combined = {
    "binary_output": v_binary_labels,
    "multilabel_output": v_labels,
}


# Навчання моделі
history = model_1.fit(
    t_data,
    t_labels_combined,
    validation_data=(v_data, v_labels_combined),
    epochs=5,  
    batch_size=32
)


Epoch 1/5
 39/200 ━━━━━━━━━━━━━━━━━━━━ 17:09 6s/step - binary_output_accuracy: 0.8493 - loss: 0.5246 - multilabel_output_accuracy: 0.4746 - multilabel_output_f1_metric: 0.0823 - multilabel_output_precision: 0.0442 - multilabel_output_recall: 0.7420

KeyboardInterrupt: 

**Друга версія моделі**

***ПРИМІТКА:***

Другий підхід не вдалось реалізувати через вимоги бібліотеки Керас. 

Згідно із логікою запропонованої архітектури модель повина мати два окремі входи: 
* повний набір даних із бінарними мітками для бінарної голови
* частковий набір даних лише із токсичними коментарями та мультимітками для мультиміткової голови і такий набор даних очевидно коротший за перший.

Проте бібіліотека Keras вимагає, щоб всі вхідні дані та вихідні мітки мали однакову довжину. 

Наступний код поданий у закоментованому вигляді для референсу. 


In [24]:
"""
# Вхідні дані для двох голов (окремі входи для кожної голови моделі)
binary_input_ids = Input(shape=(128,), dtype=tf.int32, name="binary_input_ids")
binary_attention_mask = Input(shape=(128,), dtype=tf.int32, name="binary_attention_mask")

multilabel_input_ids = Input(shape=(128,), dtype=tf.int32, name="multilabel_input_ids")
multilabel_attention_mask = Input(shape=(128,), dtype=tf.int32, name="multilabel_attention_mask")

# BERT шар для обох голов 
bert_outputs_binary = BertLayer(trainable=False)([binary_input_ids, binary_attention_mask])
bert_outputs_multilabel = BertLayer(trainable=False)([multilabel_input_ids, multilabel_attention_mask])

# Пулінг для першої голови
pooled_output_binary = GlobalAveragePooling1D()(bert_outputs_binary)
binary_dense = Dense(128, activation="swish")(pooled_output_binary)
binary_dropout = Dropout(0.3)(binary_dense)
binary_output = Dense(1, activation="sigmoid", name="binary_output")(binary_dropout)

# Пулінг для другої голови
pooled_output_multilabel = GlobalAveragePooling1D()(bert_outputs_multilabel)
multilabel_dense = Dense(128, activation="swish")(pooled_output_multilabel)
multilabel_dropout = Dropout(0.3)(multilabel_dense)
multilabel_output = Dense(6, activation="sigmoid", name="multilabel_output")(multilabel_dropout)

# Модель
model_2 = Model(
    inputs=[
        binary_input_ids, 
        binary_attention_mask, 
        multilabel_input_ids, 
        multilabel_attention_mask
    ],
    outputs=[binary_output, multilabel_output]
)

class_weights_tensor = tf.constant(class_weights, dtype=tf.float32)

# Компіляція моделі
model_2.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss={
        "binary_output": "binary_crossentropy",
        "multilabel_output": weighted_binary_crossentropy(class_weights_tensor)
    },
    loss_weights={
        "binary_output": 0.5,
        "multilabel_output": 1.0
    },
    metrics={
        "binary_output": ["accuracy"],
        "multilabel_output": ["accuracy", Precision(name="precision"), Recall(name="recall"), f1_metric]
    }
)

model_2.summary()
"""

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ binary_input_ids          │ (None, 128)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ binary_attention_mask     │ (None, 128)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multilabel_input_ids      │ (None, 128)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multilabel_attention_mask │ (None, 128)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_layer_3 (BertLayer)  │ (None, 128, 768)       │              0 │ binary_input_ids[0][0… │
│                           │                        │                │ binary_attention_mask… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_layer_4 (BertLayer)  │ (None, 128, 768)       │              0 │ multilabel_input_ids[… │
│                           │                        │                │ multilabel_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 768)            │              0 │ bert_layer_3[0][0]     │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 768)            │              0 │ bert_layer_4[0][0]     │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 128)            │         98,432 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 128)            │         98,432 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_6 (Dropout)       │ (None, 128)            │              0 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_7 (Dropout)       │ (None, 128)            │              0 │ dense_7[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ binary_output (Dense)     │ (None, 1)              │            129 │ dropout_6[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multilabel_output (Dense) │ (None, 6)              │            774 │ dropout_7[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 197,767 (772.53 KB)

 Trainable params: 197,767 (772.53 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
"""
# Генерація міток для першої голови (binary_output) - без змін у порівнянні із першою версією
t_binary_labels = np.where(np.all(t_labels == 0, axis=1), 1, 0).astype('float32')
v_binary_labels = np.where(np.all(v_labels == 0, axis=1), 1, 0).astype('float32')
"""

In [26]:
"""
# Підготовка даних для двух голов (змінено для другої версії моделі)

# Видбірка токсичних коментарів для другої голови
t_toxic_indices = np.any(t_labels == 1, axis=1)
v_toxic_indices = np.any(v_labels == 1, axis=1)

# Вхідні дані лише для токсичних прикладів
t_toxic_input_ids = t_input_ids[t_toxic_indices]
t_toxic_attention_mask = t_attention_mask[t_toxic_indices]
t_toxic_labels = t_labels[t_toxic_indices]

v_toxic_input_ids = v_input_ids[v_toxic_indices]
v_toxic_attention_mask = v_attention_mask[v_toxic_indices]
v_toxic_labels = v_labels[v_toxic_indices]

# Підготовка даних для двох голов
# Перша голова отримує повну вибірку
t_data_binary = {
    "input_ids": t_input_ids,
    "attention_mask": t_attention_mask,
}

v_data_binary = {
    "input_ids": v_input_ids,
    "attention_mask": v_attention_mask,
}

# Друга голова отримує лише токсичні приклади
t_data_multilabel = {
    "input_ids": t_toxic_input_ids,
    "attention_mask": t_toxic_attention_mask,
}

v_data_multilabel = {
    "input_ids": v_toxic_input_ids,
    "attention_mask": v_toxic_attention_mask,
}

# Підготовка міток для двох голов
t_labels_combined = {
    "binary_output": t_binary_labels,
    "multilabel_output": t_toxic_labels,
}

v_labels_combined = {
    "binary_output": v_binary_labels,
    "multilabel_output": v_toxic_labels,
}
"""

In [27]:
"""
history = model_2.fit(
    {
        "binary_input_ids": t_input_ids,  # Повна вибірка для першої голови
        "binary_attention_mask": t_attention_mask,
        "multilabel_input_ids": t_toxic_input_ids,  # Токсичні дані для другої голови
        "multilabel_attention_mask": t_toxic_attention_mask,
    },
    {
        "binary_output": t_binary_labels,  # Повні мітки для першої голови
        "multilabel_output": t_toxic_labels,  # Мітки лише для токсичних прикладів
    },
    validation_data=(
        {
            "binary_input_ids": v_input_ids,  # Повна вибірка для першої голови
            "binary_attention_mask": v_attention_mask,
            "multilabel_input_ids": v_toxic_input_ids,  # Токсичні дані для другої голови
            "multilabel_attention_mask": v_toxic_attention_mask,
        },
        {
            "binary_output": v_binary_labels,  # Повні мітки для першої голови
            "multilabel_output": v_toxic_labels,  # Мітки лише для токсичних прикладів
        }
    ),
    epochs=5,
    batch_size=32
)
"""

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 6383, 6383, 633, 633
'y' sizes: 6383, 633


**Третя версія моделі (дві послідовні моделі)**

In [28]:
# Модель для бінарної класифікації, навчена на повних даних 

# Вхідні дані
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# BERT шар
bert_outputs = BertLayer(trainable=False)([input_ids, attention_mask])

# Пулінг
pooled_output = GlobalAveragePooling1D()(bert_outputs)

# бінарна класифікація
binary_dense = Dense(128, activation="swish")(pooled_output)
binary_dropout = Dropout(0.3)(binary_dense)
binary_output = Dense(1, activation="sigmoid", name="binary_output")(binary_dropout)

# Модель
model_3_1 = Model(
    inputs=[input_ids, attention_mask],
    outputs=[binary_output]
)

# Компіляція моделі
model_3_1.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model_3_1.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_ids (InputLayer)    │ (None, 128)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_mask            │ (None, 128)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_layer_5 (BertLayer)  │ (None, 128, 768)       │              0 │ input_ids[0][0],       │
│                           │                        │                │ attention_mask[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 768)            │              0 │ bert_layer_5[0][0]     │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 128)            │         98,432 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_8 (Dropout)       │ (None, 128)            │              0 │ dense_8[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ binary_output (Dense)     │ (None, 1)              │            129 │ dropout_8[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 98,561 (385.00 KB)

 Trainable params: 98,561 (385.00 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
# Генерація міток для бінарної моделі
t_binary_labels = np.where(np.all(t_labels == 0, axis=1), 1, 0).astype('float32')
v_binary_labels = np.where(np.all(v_labels == 0, axis=1), 1, 0).astype('float32')

In [30]:
# Навчання моделі
history_3_1 = model_3_1.fit(
    t_data,
    t_binary_labels,
    validation_data=(v_data, v_binary_labels),
    epochs=5,  
    batch_size=32
)

Epoch 1/5
 13/200 ━━━━━━━━━━━━━━━━━━━━ 20:01 6s/step - accuracy: 0.6754 - loss: 0.5965

KeyboardInterrupt: 

In [31]:
# Модель для мультиміткової класифікації, навчена на токсичних коментарях 

# Вхідні дані
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# BERT шар
bert_outputs = BertLayer(trainable=False)([input_ids, attention_mask])

# Пулінг
pooled_output = GlobalAveragePooling1D()(bert_outputs)

# багатоміткова класифікація токсичних коментарів
multilabel_dense = Dense(128, activation="swish")(pooled_output)
multilabel_dropout = Dropout(0.3)(multilabel_dense)
multilabel_output = Dense(6, activation="sigmoid", name="multilabel_output")(multilabel_dropout)

# Модель
model_3_2 = Model(
    inputs=[input_ids, attention_mask],
    outputs=[multilabel_output]
)

class_weights_tensor = tf.constant(class_weights, dtype=tf.float32)

# Компіляція моделі
model_3_2.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss=weighted_binary_crossentropy(class_weights_tensor),
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), f1_metric]
)

model_3_2.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_ids (InputLayer)    │ (None, 128)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_mask            │ (None, 128)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_layer_6 (BertLayer)  │ (None, 128, 768)       │              0 │ input_ids[0][0],       │
│                           │                        │                │ attention_mask[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 768)            │              0 │ bert_layer_6[0][0]     │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 128)            │         98,432 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_9 (Dropout)       │ (None, 128)            │              0 │ dense_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multilabel_output (Dense) │ (None, 6)              │            774 │ dropout_9[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 99,206 (387.52 KB)

 Trainable params: 99,206 (387.52 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
# Підготовка даних для мультиміткової моделі (лише токсичні коментарі)

# Видбірка токсичних коментарів
t_toxic_indices = np.any(t_labels == 1, axis=1)
v_toxic_indices = np.any(v_labels == 1, axis=1)

# Вхідні дані лише для токсичних прикладів
t_toxic_input_ids = t_input_ids[t_toxic_indices]
t_toxic_attention_mask = t_attention_mask[t_toxic_indices]
t_toxic_labels = t_labels[t_toxic_indices]

v_toxic_input_ids = v_input_ids[v_toxic_indices]
v_toxic_attention_mask = v_attention_mask[v_toxic_indices]
v_toxic_labels = v_labels[v_toxic_indices]

In [33]:
# Навчання моделі
history_3_2 = model_3_2.fit(
    {
        'input_ids': t_toxic_input_ids,
        'attention_mask': t_toxic_attention_mask
    },
    t_toxic_labels,
    validation_data=(
        {
            'input_ids': v_toxic_input_ids,
            'attention_mask': v_toxic_attention_mask
        },
        v_toxic_labels),
    epochs=5,  
    batch_size=32
)

Epoch 1/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 178s 8s/step - accuracy: 0.1735 - f1_metric: 0.6451 - loss: 3.1552 - precision: 0.5597 - recall: 0.7669 - val_accuracy: 0.9091 - val_f1_metric: 0.7681 - val_loss: 2.4939 - val_precision: 0.6788 - val_recall: 0.9180
Epoch 2/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 197s 8s/step - accuracy: 0.7968 - f1_metric: 0.7521 - loss: 2.8255 - precision: 0.6505 - recall: 0.8938 - val_accuracy: 0.9091 - val_f1_metric: 0.7681 - val_loss: 2.4207 - val_precision: 0.6788 - val_recall: 0.9180
Epoch 3/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 160s 8s/step - accuracy: 0.8848 - f1_metric: 0.7509 - loss: 2.7497 - precision: 0.6526 - recall: 0.8891 - val_accuracy: 0.9455 - val_f1_metric: 0.7681 - val_loss: 2.3992 - val_precision: 0.6788 - val_recall: 0.9180
Epoch 4/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 160s 8s/step - accuracy: 0.9493 - f1_metric: 0.7555 - loss: 2.7951 - precision: 0.6578 - recall: 0.8906 - val_accuracy: 0.9455 - val_f1_metric: 0.7681 - val_loss: 2.3795 - val_precision: 0.6788 - val_recall:

**Порівняння точності двох моделей та виконання прогнозів**

Для прогнозування і оцінки тестових моделей використаємо валідаційну міні вибірку:

v_input_ids, v_attention_mask, v_labels

In [ ]:
# Оцінка першої версії моделі

# Додання класу нетоксичних коментарів (всі нулі)
all_zeros_class = np.all(v_labels == 0, axis=1).astype(int)  
y_test_expanded = np.hstack((v_labels, all_zeros_class.reshape(-1, 1)))  

# Отримання прогнозів
predictions_test = model_1.predict(
    {'input_ids': v_input_ids, 'attention_mask': v_attention_mask},
    batch_size=64
)

# Розділення прогнозів по головам
binary_output = predictions_test[0]  
multilabel_output = predictions_test[1]  

# Перетворення бінарного виходу
binary_predictions = (binary_output > 0.5).astype(int)  # Перетворення в 0 або 1

# Перетворення мультиміткового виходу
multilabel_predictions = (multilabel_output > 0.5).astype(int)  # 

# Формування вектору результатів
final_predictions = []
for binary, multilabel in zip(binary_predictions, multilabel_predictions):
    if binary == 1:
        # Якщо binary_output = 1, то всі інші мітки = 0
        final_predictions.append([0, 0, 0, 0, 0, 0, 1])  # Індекс 6 для binary_output
    else:
        # Якщо binary_output = 0, то використовуємо multilabel_output
        multilabel_result = multilabel.tolist() + [0]  # Додаємо 0 замість binary_output
        final_predictions.append(multilabel_result)

final_predictions = np.array(final_predictions)

# Оцінка моделі
print("\nКласифікаційний звіт для першого підходу:\n")
print(classification_report(y_test_expanded, final_predictions, target_names=[
    "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate", "non_toxic"
]))

# Побудова багатоміткової матриці помилок
conf_matrices = multilabel_confusion_matrix(y_test_expanded, final_predictions)

# Приклад виводу (наприклад для "toxic")
print("Confusion matrix for 'toxic':")
print(conf_matrices[0])

In [ ]:
# Оцінка другої версії моделі

# Додання класу нетоксичних коментарів (всі нулі)
all_zeros_class = np.all(v_labels == 0, axis=1).astype(int)  
y_test_expanded = np.hstack((v_labels, all_zeros_class.reshape(-1, 1)))  

# Отримання прогнозів для першої моделі
binary_predictions = model_3_1.predict(
    {'input_ids': v_input_ids, 'attention_mask': v_attention_mask},
    batch_size=64
)
binary_predictions = (binary_output > 0.5).astype(int)  # Перетворення в 0 або 1

final_predictions = []

# Прододимо по кодному прикладу валідаційних даних
for i in range(len(v_input_ids)):
    binary_prediction = binary_predictions[i]  # Прогноз бінарної моделі для поточного приклада

    if binary_prediction == 1:
        # Якщо коментар не токсичний, формуємо фінальний вектор
        final_predictions.append([0, 0, 0, 0, 0, 0, 1])  # Всі нулі + 1 на останьому індексі
    else:
        # Якщо коментар токсичний, формуємо прогноз мультимітковою моделлю
        toxic_input_ids = v_input_ids[i].reshape(1, -1)  # Приклад в форматі (1, 128)
        toxic_attention_mask = v_attention_mask[i].reshape(1, -1)

        # Прогноз мультимітковою моделлю
        multilabel_prediction = model_3_2.predict(
            {'input_ids': toxic_input_ids, 'attention_mask': toxic_attention_mask},
            batch_size=1
        )

        # Перетворення прогнозів
        multilabel_result = (multilabel_prediction > 0.5).astype(int).flatten().tolist()
        multilabel_result.append(0)  # Дадаємо 0 в останній індекс 

        # Додаємо результат у фінальні прогнози
        final_predictions.append(multilabel_result)

# Перетворення фінальних прогнозів в numpy-масив
final_predictions = np.array(final_predictions)

# Оцінка моделі
print("\nКласифікаційний звіт для другого підходу моделі:\n")
print(classification_report(y_test_expanded, final_predictions, target_names=[
    "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate", "non_toxic"
]))

# Побудова багатоміткової матриці помилок
conf_matrices = multilabel_confusion_matrix(y_test_expanded, final_predictions)

# Приклад виводу (наприклад для "toxic")
print("Confusion matrix for 'toxic':")
print(conf_matrices[0])